# video sicing for ImageMatching

- query frame :
    - 3초 단위로 slicing한 frame사용 <br>
    - 학습 이미지(metadataset)과 동일한 (512,512) 사이즈 혹은 원본 그대로 사용   
- (metadataset) searchspace
    - 전체 동영상을 프레임단위로 slicing
    - 15프레임 (60프레임 동영상 기준, 초당 4장) + 학습을 위한 (512,512)resizing
    

In [3]:
import os
import cv2

사용할 동영상의 기본 정보 확인 

In [4]:
video_path = '/USER/final_project/data/video/section2/[02]Left_01.MP4' #your data path

In [5]:
cap = cv2.VideoCapture(video_path)
cap.get(cv2.CAP_PROP_FPS)

59.94005994005994

## Make query frame from video

### 1. (0~60) 프레임 단위로 슬라이싱

In [119]:
def extract_and_save_frames_ff(video_path, target_fps, output_folder):
    # 비디오 파일을 읽음
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return
    
    # 원본 비디오의 FPS를 가져옴
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = round(original_fps / target_fps)

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        
        # 더 이상 프레임이 없으면 반복문 종료
        if not ret:
            break
        
        # 지정한 간격으로 프레임을 추출 및 저장
        if frame_count % frame_interval == 0:
            # 파일명 생성 (예: frame_0.jpg, frame_1.jpg, ...)
            filename = f"frame_{frame_count // frame_interval}.jpg"
            filepath = os.path.join(output_folder, filename)

            # 프레임 저장
            frame = cv2.resize(frame, (512,512))  # 512,512로 resizing for feature extractor model(ConvAutoEncoder)
            cv2.imwrite(filepath, frame)
        
        frame_count += 1

    # 자원 해제
    cap.release()



In [ ]:
# 함수 사용 예시
# 'example_video.mp4'는 비디오 파일의 경로
# 5는 추출하고자 하는 대상 FPS
# 'output_frames'는 저장할 폴더 경로
extract_and_save_frames_ff('example_video.mp4', 5, 'output_frames')

### 2. 초단위로 Slicing

In [120]:
import cv2
import os

def extract_and_save_frames_fs(video_path, seconds_interval, output_folder):
    # 비디오 파일을 읽음
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return
    
    # 원본 비디오의 FPS를 가져옴
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps * seconds_interval)

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        
        # 더 이상 프레임이 없으면 반복문 종료
        if not ret:
            break
        
        # 지정한 간격으로 프레임을 추출 및 저장
        if frame_count % frame_interval == 0:
            # 파일명 생성 (예: frame_0.jpg, frame_1.jpg, ...)
            filename = f"frame_{frame_count // frame_interval}.jpg"
            filepath = os.path.join(output_folder, filename)
            
            ### Resize option
            frame = cv2.resize(frame, (512,512))  # 512,512로 resizing for feature extractor model(ConvAutoEncoder)

            # 프레임 저장
            cv2.imwrite(filepath, frame)
        
        frame_count += 1

    # 자원 해제
    cap.release()



In [121]:
video_path = '/USER/final_project/data/video/section2/[02]Right_01.MP4'

In [123]:
# 함수 사용 예시
# 'example_video.mp4'는 비디오 파일의 경로
# 3는 추출하고자 하는 간격(초)
# '/USER/final_project/data/slice1'는 저장할 폴더 경로
extract_and_save_frames_fs(video_path, 3, '/USER/final_project/data/video_slicing/query/section2/right_01_slice_resize')

### 3. Make Train&Search data
학습과 유사도 검색을 위한 전체 동영상을 프레임단위로 슬라이싱한 데이터셋을 구성합니다.

In [6]:
##  NOT USE ##

def make_meta_searchspace(video_dir, target_fps, output_dir):
    total_video = []
    video_path = ''
    for i in sorted(os.listdir(video_dir)):
        video_path = video_dir + '/' + i
        total_video.append(os.listdir(video_path))
    print(total_video)

In [7]:
video_dir = '/USER/final_project/data/video'

In [8]:
def list_all_videos(video_dir):
    all_videos = []
    for section in sorted(os.listdir(video_dir)):
        section_path = os.path.join(video_dir, section)
        if os.path.isdir(section_path):
            for file in os.listdir(section_path):
                file_path = os.path.join(section_path, file)
                if os.path.isfile(file_path):
                    all_videos.append(file_path)
    return all_videos


In [9]:
video_dir = '/USER/final_project/data/video'  # video 폴더의 경로
videos = list_all_videos(video_dir)

In [10]:
type(os.path.basename('/USER/final_project/data/video/section1/[01]left_01.MP4'))

str

In [11]:
os.path.basename(video_path)

'[02]Left_01.MP4'

In [12]:
def extract_and_save_frames(video_path, output_folder, target_fps):
    # 비디오 파일을 읽음
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # 원본 비디오의 FPS를 가져옴
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    
    if target_fps > original_fps:
        frame_interval = int(target_fps)
    else :
        frame_interval = round(original_fps / target_fps)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            filename = f"{os.path.basename(video_path)}frame_{frame_count // frame_interval}.jpg"
            filepath = os.path.join(output_folder, filename)
            frame = cv2.resize(frame, (512, 512))  # 512x512로 리사이징
            cv2.imwrite(filepath, frame)

        frame_count += 1

    cap.release()
    
    
# def process_all_videos_for_dir(video_paths, target_fps, output_root):
    # for video_path in video_paths:
    ##    각 비디오 파일의 이름으로 폴더 생성
        # video_name = os.path.basename(video_path)
        # output_folder = os.path.join(output_root, os.path.splitext(video_name)[0])
        # os.makedirs(output_folder, exist_ok=True)

        # extract_and_save_frames(video_path, target_fps, output_folder)


def process_all_videos(video_paths, output_folder, target_fps):
    for video_path in video_paths:
        extract_and_save_frames(video_path, output_folder, target_fps)

In [13]:
 ## Autoencoder로 학습시키고, query이미지가 유사한 프레임을 찾을 이미지 경로를 지정합니다.
trainspace = '/USER/final_project/data/video_slicing/trainspace'
searchspace = '/USER/final_project/data/video_slicing/searchspace'

In [14]:
print(len(videos))
videos

18


['/USER/final_project/data/video/section1/[01]left_01.MP4',
 '/USER/final_project/data/video/section1/[01]left_02.MP4',
 '/USER/final_project/data/video/section1/[01]Right_01.MP4',
 '/USER/final_project/data/video/section1/[01]Right_02.MP4',
 '/USER/final_project/data/video/section1/[01]Up_01.MP4',
 '/USER/final_project/data/video/section1/[01]Up_02.MP4',
 '/USER/final_project/data/video/section2/[02]Left_01.MP4',
 '/USER/final_project/data/video/section2/[02]Left_02.MP4',
 '/USER/final_project/data/video/section2/[02]Right_01.MP4',
 '/USER/final_project/data/video/section2/[02]Right_02.MP4',
 '/USER/final_project/data/video/section2/[02]Up_01.MP4',
 '/USER/final_project/data/video/section2/[02]Up_02.MP4',
 '/USER/final_project/data/video/section3/[03]Left_01.MP4',
 '/USER/final_project/data/video/section3/[03]Left_02.MP4',
 '/USER/final_project/data/video/section3/[03]Right_01.MP4',
 '/USER/final_project/data/video/section3/[03]Right_02.MP4',
 '/USER/final_project/data/video/section3/

In [15]:
#Query대상 비디오(2구역 왼쪽벽면 촬영 영상)리스트에서 제거
videos.remove('/USER/final_project/data/video/section2/[02]Left_01.MP4')

In [16]:
len(videos)

17

SearchSpace

In [18]:
#videos = ['/path/to/video1.mp4', '/path/to/video2.mp4']  # 비디오 파일 목록
process_all_videos(videos, searchspace, 180)  # 5는 목표 FPS, 'output_frames'는 저장할 루트 폴더

TrainSpace

In [136]:
process_all_videos(videos, trainspace, 15)

In [57]:
len(os.listdir(searchspace))

476